In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from collections import Counter
import time
data_comments_original  = pd.read_csv('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq/Dataset/teen_sampled_comments_1.csv')

In [ ]:
data_comments_original['Row No.'] = range(200000)

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Cognitive_Distortions_DrFarhanAli_Tawseeq')
import targetwords

In [ ]:
# selected_columns_data_posts_original = ['title', 'body', 'created_utc']
# reddit_posts = data_posts_original[selected_columns_data_posts_original]
n_order_unigrams = 1
n_order_bigrams = 2
n_order_trigrams = 3
n_order_quadgrams = 4
n_order_pentagrams = 5
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#tokenizing
tokenized_data_comments = [word_tokenize(str(text).lower()) for text in data_comments_original['body']]

In [ ]:
n_orders = [1, 2, 3, 4, 5]
def generate_ngrams(tokens, n_orders):
    ngram_data = {n: [] for n in n_orders}
    for n in n_orders:
        ngram_data[n] = [' '.join(ngram) for ngram in ngrams(tokens, n)]
    return ngram_data

all_ngrams = [generate_ngrams(tokens, n_orders) for tokens in tokenized_data_comments]
distortion_ngram_sets = {distortion: set(targetwords.__dict__[distortion]) for distortion in targetwords.__dict__.keys() if 'target_' in distortion}
def process_data(ngram_data_list, original_data):
    results = []
    for i, ngram_data in enumerate(ngram_data_list):
        distortion_counts = {distortion: 0 for distortion in distortion_ngram_sets.keys()}
        for n in n_orders:
            for ngram_str in ngram_data[n]:
                for distortion in distortion_counts.keys():
                    if ngram_str in distortion_ngram_sets[distortion]:
                        distortion_counts[distortion] += 1
        row = {'Comment No.': original_data.index[i]}
        row.update(distortion_counts)
        results.append(row)
    return pd.DataFrame(results)

final_results = process_data(all_ngrams, data_comments_original)

In [ ]:
final_results.sum()

Comment No.                              19999900000
target_catastrophizing                            18
target_dichotomous_Reasoning                   27251
target_disqualifying_the_positives               249
target_emotional_reasoning                        11
target_fortune_telling                            40
target_labeling_and_mislabeling                 1982
target_magnification_and_minimization           2544
target_mental_filtering                           15
target_mindreading                               283
target_overgeneralizing                          943
target_personalizing                             552
target_should_statements                        5491
dtype: int64

In [ ]:
def generate_ngrams_with_timestamps(tokens_list, n_order, timestamps):
    ngram_data = []
    for i, tokens in enumerate(tokens_list):
        ngrams_list = list(ngrams(tokens, n_order))
        for ngram in ngrams_list:
            ngram_data.append((ngram, timestamps[i], i))  
    return ngram_data

timestamps = data_comments_original['created_utc']

unigram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_unigrams, timestamps)
bigram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_bigrams, timestamps)
trigram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_trigrams, timestamps)
quadgram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_quadgrams, timestamps)
pentagram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_pentagrams, timestamps)

In [ ]:
covid_start = 1579478400  
covid_end = 1594598400    
end_date = 1697372672

before_covid = data_comments_original[data_comments_original['created_utc'] < covid_start]
during_covid = data_comments_original[(data_comments_original['created_utc'] >= covid_start) & (data_comments_original['created_utc'] <= covid_end)]
after_covid = data_comments_original[(data_comments_original['created_utc'] > covid_end) & (data_comments_original['created_utc'] <= end_date)]

nltk.download('punkt')
tokenized_before_covid_body = [word_tokenize(str(text).lower()) for text in before_covid['body']]
tokenized_during_covid_body = [word_tokenize(str(text).lower()) for text in during_covid['body']]
tokenized_after_covid_body = [word_tokenize(str(text).lower()) for text in after_covid['body']]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
target_words = {distortion: set(targetwords.__dict__[distortion]) for distortion in targetwords.__dict__.keys() if 'target_' in distortion}

def process_data(tokenized_bodies, original_data):
    results = []
    for i, body_tokens in enumerate(tokenized_bodies):
        distortion_counts = {distortion: 0 for distortion in target_words}
        for token in body_tokens:
            for distortion, words_set in target_words.items():
                if token in words_set:
                    distortion_counts[distortion] += 1
        row = {'Row No.': original_data.iloc[i]['Row No.']}
        row.update(distortion_counts)
        results.append(row)
    return pd.DataFrame(results)

before_covid_results = process_data(tokenized_before_covid_body, before_covid)
during_covid_results = process_data(tokenized_during_covid_body, during_covid)
after_covid_results = process_data(tokenized_after_covid_body, after_covid)

In [ ]:
before_covid_results.to_csv('your_path', index=False)
during_covid_results.to_csv('your_path', index=False)
after_covid_results.to_csv('your_path', index=False)

In [ ]:
def generate_ngrams_with_timestamps(tokens_list, n_order, timestamps):
    ngram_data = [ngrams(tokens, n_order) for tokens in tokens_list]
    ngrams_with_timestamps = []

    for i, ngrams_list in enumerate(ngram_data):
        for ngram in ngrams_list:
            ngrams_with_timestamps.append((ngram, timestamps[i]))

    return ngrams_with_timestamps

timestamps = data_comments_original['created_utc']

unigram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_unigrams, timestamps)
bigram_data= generate_ngrams_with_timestamps(tokenized_data_comments, n_order_bigrams, timestamps)
trigram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_trigrams, timestamps)
quadgram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_quadgrams, timestamps)
pentagram_data = generate_ngrams_with_timestamps(tokenized_data_comments, n_order_pentagrams, timestamps)


In [ ]:
print(np.max(data_comments_original['created_utc']))
print(np.min(data_comments_original['created_utc']))

1706750948.0
1546274545.0


In [ ]:
import time
weeks = np.zeros(267)

#target_1gram = ['only', 'every', 'everyone', 'everybody', 'everything', 'everywhere', 'always', 'perfect', 'all', 'nobody'] #this on is for test purpose

def count_occurrences_for_target_1gram(flat_unigrams, target_word):
    start_time = time.time()
    count = 0
    for i in range(len(flat_unigrams)):
      #current_word = str(flat_unigrams[i])
      current_word = ' '.join(map(str, flat_unigrams[i][0]))
      #print(target_word)
      if current_word.lower() == target_word.lower():
        count += 1
        index = np.floor((1706750948.0 - flat_unigrams[i][1])/604800)
        index = int(index)
        weeks[index] +=1
    end_time = time.time()
    return count, end_time - start_time


#for target_word in targetwords.target_dichotomous_Reasoning:
for target_word in targetwords.target_mindreading:
#for target_word in targetwords.target_disqualifying_the_positives:
#for target_word in targetwords.target_emotional_reasoning:
# for target_word in targetwords.target_catastrophizing:
# for target_word in targetwords.target_fortune_telling:
#for target_word in targetwords.target_labeling_and_mislabeling:
# for target_word in targetwords.target_magnification_and_minimization:
# for target_word in targetwords.target_mental_filtering:
# for target_word in targetwords.target_overgeneralizing:
# for target_word in targetwords.target_personalizing:
#for target_word in targetwords.target_should_statements:

    count1, time_taken1 = count_occurrences_for_target_1gram(unigram_data, target_word)
    count2, time_taken2 = count_occurrences_for_target_1gram(bigram_data, target_word)
    count3, time_taken3 = count_occurrences_for_target_1gram(trigram_data, target_word)
    count4, time_taken4 = count_occurrences_for_target_1gram(quadgram_data, target_word)
    count5, time_taken5 = count_occurrences_for_target_1gram(pentagram_data, target_word)
    print(f"Occurrences of the target 1-gram '{target_word}' in body: {count1+count2+count3+count4+count5}")
    print(f"Time taken: {time_taken1+time_taken2+time_taken3+time_taken4+time_taken5} seconds")
    print("---")


Occurrences of the target 1-gram 'everyone believes' in body: 0
Time taken: 11.565910577774048 seconds
---
Occurrences of the target 1-gram 'every1 believes' in body: 0
Time taken: 11.304389715194702 seconds
---
Occurrences of the target 1-gram 'everyone knows' in body: 23
Time taken: 8.207172870635986 seconds
---
Occurrences of the target 1-gram 'every1 knows' in body: 0
Time taken: 11.53499460220337 seconds
---
Occurrences of the target 1-gram 'everyone thinks' in body: 9
Time taken: 10.55268144607544 seconds
---
Occurrences of the target 1-gram 'every1 thinks' in body: 0
Time taken: 8.355299949645996 seconds
---
Occurrences of the target 1-gram 'everyone will believe' in body: 0
Time taken: 10.933498620986938 seconds
---
Occurrences of the target 1-gram 'everyone'll believe' in body: 0
Time taken: 10.368813037872314 seconds
---
Occurrences of the target 1-gram 'every1 will believe' in body: 0
Time taken: 8.081238746643066 seconds
---
Occurrences of the target 1-gram 'everyone will k

In [ ]:
weeks
sumw = np.sum(weeks)
print(sumw)
#week 55 is the covid start date in singapore

283.0


In [ ]:
weeks

array([0., 0., 0., 0., 0., 0., 0., 2., 0., 2., 2., 0., 1., 1., 2., 1., 1.,
       0., 2., 4., 4., 5., 0., 2., 1., 1., 2., 1., 0., 1., 1., 0., 2., 1.,
       1., 1., 1., 2., 0., 1., 2., 0., 1., 2., 1., 1., 1., 1., 0., 2., 1.,
       1., 0., 2., 2., 0., 2., 2., 1., 0., 0., 1., 0., 0., 1., 3., 1., 1.,
       1., 1., 1., 0., 2., 2., 0., 1., 1., 2., 2., 2., 0., 1., 1., 1., 1.,
       2., 2., 1., 2., 2., 0., 1., 3., 0., 3., 2., 1., 2., 2., 1., 2., 1.,
       4., 1., 1., 2., 2., 0., 5., 1., 0., 2., 1., 0., 2., 0., 2., 1., 0.,
       0., 2., 1., 1., 0., 3., 3., 0., 2., 1., 1., 2., 0., 5., 0., 0., 0.,
       1., 0., 0., 1., 0., 2., 1., 0., 2., 0., 2., 0., 5., 1., 2., 3., 1.,
       1., 1., 0., 0., 3., 0., 1., 2., 0., 2., 1., 0., 1., 0., 1., 4., 0.,
       4., 1., 1., 0., 2., 2., 3., 1., 0., 2., 3., 1., 0., 2., 0., 1., 0.,
       2., 3., 0., 0., 1., 1., 1., 0., 1., 2., 1., 0., 0., 0., 4., 1., 0.,
       0., 0., 4., 2., 1., 0., 0., 0., 1., 0., 0., 0., 0., 4., 0., 0., 1.,
       0., 1., 0., 0., 0.